In [1]:
from flashtext import KeywordProcessor
import time
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

terms = ['agriculture','agricultural','agritech','agtech','agronomy','farming','gardening','agronomics','agrology','horticulture','agribusiness','cultivation', 'forestry', 'animal husbandry', 'hunting', 'fishing', 'planting', 'sowing', 'fertisiling', 'harvesting', 'mowing','aviculture', 'apiculture', 'pisciculture', 'sowing', 'planting', 'manuring', 'harvesting']
print('terms to use: {}'.format(len(terms)))
litigation_cases = pd.read_csv('datasets/litigation_cases_06072020.csv')
litigation_cases.head(4)

terms to use: 28


,Title,Geography,Geography ISO,Jurisdiction,Citation Reference Number,Responses,At issue,Connected Internal Laws,Connected External Laws,Sectors,Side A,Side B,Side C,Events,Summary
0,Mapuche Confederation of Neuquén v. YPF et al.,Argentina,ARG,Argentina,NaN,Mitigation,<div>Indigenous group filed criminal complaint...,NaN,NaN,NaN,Mapuche Confederation of Neuqu̩n|ngo,"Subsecretary of the Environment, the Provision...",NaN,31/10/2018|Case opened||,<div>An indigenous group in Argentina is suing...
1,Hunter Environment Lobby Inc. v. Minister for ...,Australia,AUS,Australia,[2011] NSWLEC 221; [2012] NSWLEC 40,Mitigation,Challenge to conditions imposed on approval of...,NaN,NaN,NaN,Hunter Community Environment Centre Inc.|ngo,Minister for Planning and Delta Electricity|go...,NaN,25/12/2011|Case opened,The case relates to the approval of diverse ex...
2,Charles & Howard Pty. Ltd. v. Redland Shire Co...,Australia,AUS,Australia,[2007] QCA 200,Adaptation,<div>Challenge to city council's decision limi...,NaN,NaN,NaN,Charles & Howard Pty Ltd|corporation,Redland Shire Council|government,NaN,25/12/2008|Case opened||,<div>The owner of property located on Australi...
3,Spencer v. Commonwealth (High Court of Austral...,Australia,AUS,Australia,[2010] HCA 28,Mitigation,<div>Appeal of dismissal of challenge to statu...,NaN,NaN,NaN,Peter Spencer|individual,Commonwealth of Australia|government,NaN,25/12/2010|Case opened||,<div>The appellant challenged two state laws t...


---
## Tagging CCM Litigation cases related to Agriculture

In [2]:
def extract(vec, dictionary, info=False):
    matrix = []
    for line in vec:
        matrix.append(dictionary.extract_keywords(str(line), span_info=info))
    return matrix

In [3]:
litigation_cases['sector_agriculture'] = litigation_cases.Sectors.apply(lambda x: (1 if 'Agriculture' in str(x) else 0))
litigation_cases.sector_agriculture.sum()

1

In [4]:
terms_dict = KeywordProcessor()
terms_dict.add_keywords_from_list(terms)
terms_extracted = extract(litigation_cases.Summary, terms_dict)
rows = [list(set(i)) if len(i)>0 else '' for i in terms_extracted]
litigation_cases['matches'] = [str(i).replace('[', '').replace(']', '') for i in rows]
litigation_cases['count_matches'] = [len(i) for i in terms_extracted]


litigation_cases['agriculture_litigation'] = litigation_cases['count_matches']+litigation_cases['sector_agriculture']
print(litigation_cases.agriculture_litigation.sum())

25


---
## Subsetting dataset to agriculture related cases in EU

In [5]:
# selecting the relevant countries
# data_laws.Geography.unique()
EU=['Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','European Union','Finland', 'France','Germany','Greece','Hungary', 'Ireland', 'Italy','Latvia','Lithuania','Luxembourg','Malta', 'Netherlands','Poland', 'Portugal','Romania','Slovakia', 'Slovenia', 'Spain', 'Sweden','United Kingdom',]

# creating dummy for EU country (individual countries + European Union)
EU_country= []
for i in litigation_cases.Geography:
    country =  1 if i in EU else 0
    EU_country.append(country)
    
# adding the dummy to the original dataset
litigation_cases['EU_country']=EU_country
litigation_cases.EU_country.sum()

170

In [6]:
# subset dataset with only agriculture litigation cases
litigation_cases_subset=litigation_cases[litigation_cases["agriculture_litigation"]>0] 
litigation_cases_subset=litigation_cases_subset.sort_values(by=['agriculture_litigation'], ascending=False)

# subset dataset with only EU countries
litigation_cases_subset=litigation_cases_subset[litigation_cases_subset["EU_country"]>0] 
litigation_cases_subset=litigation_cases_subset.sort_values(by=['EU_country'], ascending=False)

---
## Save as Excel file

In [7]:
litigation_cases_subset.sort_values('count_matches', ascending=False).to_excel('datasets/litigation_cases_AGRI.xlsx')
litigation_cases_subset.head()

,Title,Geography,Geography ISO,Jurisdiction,Citation Reference Number,Responses,At issue,Connected Internal Laws,Connected External Laws,Sectors,Side A,Side B,Side C,Events,Summary,sector_agriculture,matches,count_matches,agriculture_litigation,EU_country
142,Agrargenossenschaft Neuzelle eG v. Landrat des...,European Union,EUR,European Union,[2013] Case C-545/11,Adaptation,Preliminary ruling on two 2009 amendments to a...,NaN,NaN,NaN,Agrargenossenschaft Neuzelle eG|corporation,Head of Administrative Services of Oder-Spree ...,NaN,25/12/2013|Case opened,This case was a request for a preliminary ruli...,0,"'agriculture', 'agricultural'",2,2,1
192,Armando Ferrao Carvalho and Others v. The Euro...,European Union,EUR,European Union,Case no. T-330/18,Mitigation,<div>Action seeking an injunction to order the...,EU Emission Trading Scheme (EU ETS) (Directive...,NaN,Economy-wide,Armando Ferrao Carvalho and Others|individual,The European Parliament and the Council|govern...,NaN,23/05/2018|case opened||;08/05/2019|case decid...,"<div>Ten families, including children, from Po...",0,"'agriculture', 'forestry'",2,2,1
200,External Contribution to the French Constituti...,France,FRA,France,NaN,Mitigation,Environmental group filed memorandum asking Fr...,NaN,NaN,NaN,Notre Affaire à Tous|ngo,NaN,NaN,26/09/2019|Law of interest passed|Parlament ad...,"On September 26, 2019, the French Parliament a...",0,'agriculture',1,1,1
374,Vince et al. v. Secretary of State for Busines...,United Kingdom,GBR,United Kingdom,NaN,Mitigation,<div>Plaintiffs sought review of energy nation...,Climate Change Act|http://climate-laws.org/ccl...,Planning Act 2008|http://www.legislation.gov.u...,Energy,Dale Vince|individual;George Monbiot|individua...,United Kingdom|government,NaN,18/05/2020|Judicial review sought||,"<div>On May 18, 2020, three plaintiffs sought ...",0,'farming',1,1,1
